In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import nltk

In [4]:
data = pd.read_csv('text2sql.csv')

In [5]:
len(data)

3072

### Data CleanUp

* Sequence of lower words

In [6]:
data['seq1'] = data['seq1'].str.lower()
data['seq2'] = data['seq2'].str.lower()
data['seq3'] = data['seq3'].str.lower()
data['sql1'] = data['sql1'].str.lower()
data['sql2'] = data['sql2'].str.lower()
data['sql3'] = data['sql3'].str.lower()

In [7]:
data.head()

,seq1,seq2,seq3,sql1,sql2,sql3
0,past hour walk status,my past hour walk performance,what is my past hour walk count ?,select walk where hour is equal to one,select walk where hour is one,select walk where hour is past one
1,last hour walk status,past hour walk status,past hour walk performance,select walk where hour is equal to one,select walk where hour is one past,select walk where hour is past one
2,past two hour walk status,my past two hour walk performance,what is my two past hour walk count ?,select walk where hour is equal to sum of past...,select walk where hour sum of last two hours,select walk where hour is aggregate of past tw...
3,last two hour walk status,past two hour walk status,past two hour walk performance,select walk where hour is sum of past two hours,select walk where hour is sum of past two hour,select walk where hour is equal to sum of past...
4,past three hour walk status,my past three hour walk performance,what is my three past hour walk count ?,select walk where hour is equal to sum of past...,select walk where hour sum of last three hours,select walk where hour is aggregate of past th...


In [8]:
data['seq1'] = data['seq1'].str.replace('?','')
data['seq2'] = data['seq2'].str.replace('?','')
data['seq3'] = data['seq3'].str.replace('?','')
data['sql1'] = data['sql1'].str.replace('?','')
data['sql2'] = data['sql2'].str.replace('?','')
data['sql3'] = data['sql3'].str.replace('?','')

In [9]:
data.head()

,seq1,seq2,seq3,sql1,sql2,sql3
0,past hour walk status,my past hour walk performance,what is my past hour walk count,select walk where hour is equal to one,select walk where hour is one,select walk where hour is past one
1,last hour walk status,past hour walk status,past hour walk performance,select walk where hour is equal to one,select walk where hour is one past,select walk where hour is past one
2,past two hour walk status,my past two hour walk performance,what is my two past hour walk count,select walk where hour is equal to sum of past...,select walk where hour sum of last two hours,select walk where hour is aggregate of past tw...
3,last two hour walk status,past two hour walk status,past two hour walk performance,select walk where hour is sum of past two hours,select walk where hour is sum of past two hour,select walk where hour is equal to sum of past...
4,past three hour walk status,my past three hour walk performance,what is my three past hour walk count,select walk where hour is equal to sum of past...,select walk where hour sum of last three hours,select walk where hour is aggregate of past th...


In [10]:
data['sql1'] = 'start_ '+data['sql1'].astype(str)+' _end'
data['sql2'] = 'start_ '+data['sql2'].astype(str)+' _end'
data['sql3'] = 'start_ '+data['sql3'].astype(str)+' _end'

In [11]:
len(data['sql1']), len(data['sql2']), len(data['sql3'])

(3072, 3072, 3072)

In [49]:
# Input_Texts = Natural Language Text Sequence
# Target_Texts = Natural Language SQL Sequence

input_texts = []
target_texts = []

for column in data[['seq1', 'seq2', 'seq3']]:
    columnobj = data[column]
    input_texts.extend(columnobj.values.tolist())
    
for column in data[['sql1', 'sql2', 'sql3']]:
    columnobj = data[column]
    target_texts.extend(columnobj.values.tolist())
    
# input_texts,target_texts

# BOW Implementation

In [12]:
# import these modules 

all_seq_words=set()
all_sql_words= set()

length_list_seq = []
length_list_sql = []

sequence = data.seq1

for l in sequence:
    length_list_seq.append(len(l.split()))
#     print(l.split())
#     print(length_list)
# print(np.max(length_list))
for eng in sequence:
    for word in eng.split():
        if word not in all_seq_words:
            all_seq_words.add(word)

sequence = data.seq2 

for l in sequence:
    length_list_seq.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_seq_words:
            all_seq_words.add(word)
            
print("Maximum SEQ Length",np.max(length_list_seq))
max_seq_length = np.max(length_list_seq)

sequence = data.seq3

for l in sequence:
    length_list_seq.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_seq_words:
            all_seq_words.add(word)
    

sequence = data.sql1 

for l in sequence:
    length_list_sql.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_sql_words:
            all_sql_words.add(word)

sequence = data.sql2 

for l in sequence:
    length_list_sql.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_sql_words:
            all_sql_words.add(word)

sequence = data.sql3 

for l in sequence:
    length_list_sql.append(len(l.split()))

for eng in sequence:
    for word in eng.split():
        if word not in all_sql_words:
            all_sql_words.add(word)
                        
print("Maximum SQL Length",np.max(length_list_sql))
max_sql_length = np.max(length_list_sql)

Maximum SEQ Length 8
Maximum SQL Length 16


In [13]:
input_words = sorted(list(all_seq_words))
target_words = sorted(list(all_sql_words))

num_encoder_tokens = len(all_seq_words)
num_decoder_tokens = len(all_sql_words)

In [14]:
# input_words,target_words 
# num_encoder_tokens,num_decoder_tokens #(36, 40)

In [15]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [16]:
# input_token_index,target_token_index

In [17]:
total_length_sql = len(data['sql1']) + len(data['sql2']) + len(data['sql3'])
total_length_seq = len(data['seq1']) + len(data['seq2']) + len(data['seq3'])

In [18]:
total_length_seq,total_length_sql

(9216, 9216)

In [19]:
sql_input_size = total_length_sql*max_sql_length*num_decoder_tokens
seq_input_size = total_length_seq*max_seq_length*num_encoder_tokens

In [20]:
seq_input_size,sql_input_size

(2654208, 6193152)

In [21]:
encoder_input_data = np.zeros((total_length_seq, max_seq_length),dtype='float32')
decoder_input_data = np.zeros((total_length_sql, max_sql_length),dtype='float32')
decoder_target_data = np.zeros((total_length_sql, max_sql_length, num_decoder_tokens),dtype='float32')

In [22]:
encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape

((9216, 8), (9216, 16), (9216, 16, 42))

In [23]:
sequenceData= list()
sqlData = list()

for column in data[['seq1', 'seq2', 'seq3']]:
    columnobj = data[column]
    sequenceData.extend(columnobj.values.tolist())
    
for column in data[['sql1', 'sql2', 'sql3']]:
    columnobj = data[column]
    sqlData.extend(columnobj.values.tolist())

In [24]:
for i, (input_text, target_text) in enumerate(zip(sequenceData,sqlData)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [25]:
encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape

((9216, 8), (9216, 16), (9216, 16, 42))

In [26]:
# encoder_input_data,decoder_input_data,decoder_target_data

In [27]:
max_encoder_seq_length = max([len(txt) for txt in sequenceData])
max_decoder_seq_length = max([len(txt) for txt in sqlData])
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Max sequence length for inputs: 40
Max sequence length for outputs: 87


# Keras Encoder > Decoder Model

In [28]:
embedding_size = 50

In [29]:
import keras
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

Using TensorFlow backend.


## Encoder

In [30]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(embedding_size, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [31]:
num_encoder_tokens,encoder_inputs

(36, <tf.Tensor 'input_1:0' shape=(None, None) dtype=float32>)

## Decoder

In [32]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(embedding_size, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [33]:
num_decoder_tokens,decoder_inputs

(42, <tf.Tensor 'input_2:0' shape=(None, None) dtype=float32>)

In [34]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     1800        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     2100        input_2[0][0]                    
____________________________________________________________________________________________

## Fit the mode

In [35]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=16,
          epochs=1000,
          validation_split=0.05)

C:\Users\vijay\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8755 samples, validate on 461 samples
Epoch 1/1000
8755/8755 [==============================] - 15s 2ms/step - loss: 0.7564 - acc: 0.8013 - val_loss: 0.4168 - val_acc: 0.8643
Epoch 2/1000
8755/8755 [==============================] - 13s 1ms/step - loss: 0.2939 - acc: 0.8998 - val_loss: 0.2804 - val_acc: 0.8983
Epoch 3/1000
8755/8755 [==============================] - 11s 1ms/step - loss: 0.2214 - acc: 0.9070 - val_loss: 0.2059 - val_acc: 0.9188
Epoch 4/1000
8755/8755 [==============================] - 11s 1ms/step - loss: 0.1812 - acc: 0.8722 - val_loss: 0.1791 - val_acc: 0.8941
Epoch 5/1000
8755/8755 [==============================] - 12s 1ms/step - loss: 0.1536 - acc: 0.8629 - val_loss: 0.1489 - val_acc: 0.9283
Epoch 6/1000
8755/8755 [==============================] - 13s 1ms/step - loss: 0.1324 - acc: 0.8948 - val_loss: 0.1223 - val_acc: 0.9398
Epoch 7/1000
8755/8755 [==============================] - 12s 1ms/step - loss: 0.1121 - acc: 0.8815 - val_loss: 0.1021 - val_acc: 0

8755/8755 [==============================] - 13s 1ms/step - loss: 0.0254 - acc: 0.8758 - val_loss: 0.0171 - val_acc: 0.9389
Epoch 61/1000
8755/8755 [==============================] - 13s 1ms/step - loss: 0.0254 - acc: 0.8786 - val_loss: 0.0179 - val_acc: 0.9376
Epoch 62/1000
8755/8755 [==============================] - 13s 1ms/step - loss: 0.0255 - acc: 0.8826 - val_loss: 0.0175 - val_acc: 0.9376
Epoch 63/1000
8755/8755 [==============================] - 13s 1ms/step - loss: 0.0254 - acc: 0.8853 - val_loss: 0.0173 - val_acc: 0.9376
Epoch 64/1000
8755/8755 [==============================] - 12s 1ms/step - loss: 0.0255 - acc: 0.8920 - val_loss: 0.0171 - val_acc: 0.9450
Epoch 65/1000
8755/8755 [==============================] - 13s 1ms/step - loss: 0.0253 - acc: 0.8948 - val_loss: 0.0171 - val_acc: 0.9402
Epoch 66/1000
8755/8755 [==============================] - 12s 1ms/step - loss: 0.0254 - acc: 0.8950 - val_loss: 0.0175 - val_acc: 0.9429
Epoch 67/1000
8755/8755 [=======================

Epoch 178/1000
8755/8755 [==============================] - 15s 2ms/step - loss: 0.0248 - acc: 0.9351 - val_loss: 0.0170 - val_acc: 0.9729
Epoch 179/1000
8755/8755 [==============================] - 15s 2ms/step - loss: 0.0249 - acc: 0.9345 - val_loss: 0.0170 - val_acc: 0.9637
Epoch 180/1000
8755/8755 [==============================] - 12s 1ms/step - loss: 0.0250 - acc: 0.9263 - val_loss: 0.0169 - val_acc: 0.9677
Epoch 181/1000
8755/8755 [==============================] - 15s 2ms/step - loss: 0.0249 - acc: 0.9241 - val_loss: 0.0170 - val_acc: 0.9639
Epoch 182/1000
8755/8755 [==============================] - 15s 2ms/step - loss: 0.0249 - acc: 0.9240 - val_loss: 0.0173 - val_acc: 0.9624
Epoch 183/1000
8755/8755 [==============================] - 15s 2ms/step - loss: 0.0251 - acc: 0.9281 - val_loss: 0.0182 - val_acc: 0.9664
Epoch 184/1000
8755/8755 [==============================] - 15s 2ms/step - loss: 0.0249 - acc: 0.9283 - val_loss: 0.0172 - val_acc: 0.9680
Epoch 185/1000
8755/8755 [=

8755/8755 [==============================] - 5s 517us/step - loss: 0.0249 - acc: 0.9038 - val_loss: 0.0170 - val_acc: 0.9355
Epoch 295/1000
8755/8755 [==============================] - 5s 519us/step - loss: 0.0249 - acc: 0.9037 - val_loss: 0.0171 - val_acc: 0.9361
Epoch 296/1000
8755/8755 [==============================] - 5s 519us/step - loss: 0.0249 - acc: 0.9018 - val_loss: 0.0170 - val_acc: 0.9380
Epoch 297/1000
8755/8755 [==============================] - 5s 520us/step - loss: 0.0250 - acc: 0.9028 - val_loss: 0.0170 - val_acc: 0.9368
Epoch 298/1000
8755/8755 [==============================] - 5s 524us/step - loss: 0.0248 - acc: 0.8999 - val_loss: 0.0169 - val_acc: 0.9329
Epoch 299/1000
8755/8755 [==============================] - 5s 522us/step - loss: 0.0248 - acc: 0.8944 - val_loss: 0.0170 - val_acc: 0.9341
Epoch 300/1000
8755/8755 [==============================] - 4s 512us/step - loss: 0.0248 - acc: 0.8936 - val_loss: 0.0169 - val_acc: 0.9341
Epoch 301/1000
8755/8755 [=========

...

8755/8755 [==============================] - 5s 515us/step - loss: 0.0247 - acc: 0.8309 - val_loss: 0.0169 - val_acc: 0.8903
Epoch 527/1000
8755/8755 [==============================] - 5s 520us/step - loss: 0.0247 - acc: 0.8277 - val_loss: 0.0169 - val_acc: 0.8890
Epoch 528/1000
8755/8755 [==============================] - 5s 518us/step - loss: 0.0247 - acc: 0.8316 - val_loss: 0.0170 - val_acc: 0.8964
Epoch 529/1000
8755/8755 [==============================] - 4s 502us/step - loss: 0.0247 - acc: 0.8317 - val_loss: 0.0169 - val_acc: 0.8857
Epoch 530/1000
8755/8755 [==============================] - 4s 504us/step - loss: 0.0247 - acc: 0.8318 - val_loss: 0.0170 - val_acc: 0.8972
Epoch 531/1000
8755/8755 [==============================] - 4s 511us/step - loss: 0.0247 - acc: 0.8315 - val_loss: 0.0169 - val_acc: 0.8959
Epoch 532/1000
8755/8755 [==============================] - 5s 516us/step - loss: 0.0247 - acc: 0.8338 - val_loss: 0.0169 - val_acc: 0.8948
Epoch 533/1000
8755/8755 [=========

8755/8755 [==============================] - 5s 516us/step - loss: 0.0249 - acc: 0.8251 - val_loss: 0.0170 - val_acc: 0.8724
Epoch 643/1000
8755/8755 [==============================] - 5s 517us/step - loss: 0.0247 - acc: 0.8223 - val_loss: 0.0171 - val_acc: 0.8812
Epoch 644/1000
8755/8755 [==============================] - 5s 519us/step - loss: 0.0248 - acc: 0.8231 - val_loss: 0.0171 - val_acc: 0.8814
Epoch 645/1000
8755/8755 [==============================] - 4s 507us/step - loss: 0.0247 - acc: 0.8249 - val_loss: 0.0170 - val_acc: 0.8863
Epoch 646/1000
8755/8755 [==============================] - 4s 498us/step - loss: 0.0247 - acc: 0.8227 - val_loss: 0.0170 - val_acc: 0.8861
Epoch 647/1000
8755/8755 [==============================] - 4s 500us/step - loss: 0.0248 - acc: 0.8215 - val_loss: 0.0170 - val_acc: 0.8799
Epoch 648/1000
8755/8755 [==============================] - 4s 509us/step - loss: 0.0247 - acc: 0.8182 - val_loss: 0.0169 - val_acc: 0.8761
Epoch 649/1000
8755/8755 [=========

8755/8755 [==============================] - 5s 524us/step - loss: 0.0247 - acc: 0.8094 - val_loss: 0.0170 - val_acc: 0.8631
Epoch 759/1000
8755/8755 [==============================] - 5s 522us/step - loss: 0.0247 - acc: 0.8094 - val_loss: 0.0169 - val_acc: 0.8618
Epoch 760/1000
8755/8755 [==============================] - 5s 523us/step - loss: 0.0246 - acc: 0.8113 - val_loss: 0.0170 - val_acc: 0.8593
Epoch 761/1000
8755/8755 [==============================] - 5s 516us/step - loss: 0.0247 - acc: 0.8102 - val_loss: 0.0171 - val_acc: 0.8568
Epoch 762/1000
8755/8755 [==============================] - 5s 518us/step - loss: 0.0248 - acc: 0.8103 - val_loss: 0.0213 - val_acc: 0.8644
Epoch 763/1000
8755/8755 [==============================] - 5s 518us/step - loss: 0.0247 - acc: 0.8115 - val_loss: 0.0170 - val_acc: 0.8701
Epoch 764/1000
8755/8755 [==============================] - 5s 521us/step - loss: 0.0247 - acc: 0.8093 - val_loss: 0.0170 - val_acc: 0.8656
Epoch 765/1000
8755/8755 [=========

8755/8755 [==============================] - 5s 515us/step - loss: 0.0247 - acc: 0.8137 - val_loss: 0.0170 - val_acc: 0.8698
Epoch 875/1000
8755/8755 [==============================] - 5s 515us/step - loss: 0.0246 - acc: 0.8174 - val_loss: 0.0169 - val_acc: 0.8816
Epoch 876/1000
8755/8755 [==============================] - 5s 516us/step - loss: 0.0247 - acc: 0.8170 - val_loss: 0.0169 - val_acc: 0.8670
Epoch 877/1000
8755/8755 [==============================] - 5s 514us/step - loss: 0.0247 - acc: 0.8164 - val_loss: 0.0170 - val_acc: 0.8698
Epoch 878/1000
8755/8755 [==============================] - 5s 515us/step - loss: 0.0247 - acc: 0.8201 - val_loss: 0.0169 - val_acc: 0.8711
Epoch 879/1000
8755/8755 [==============================] - 5s 514us/step - loss: 0.0247 - acc: 0.8173 - val_loss: 0.0170 - val_acc: 0.8685
Epoch 880/1000
8755/8755 [==============================] - 4s 500us/step - loss: 0.0246 - acc: 0.8199 - val_loss: 0.0169 - val_acc: 0.8659
Epoch 881/1000
8755/8755 [=========

8755/8755 [==============================] - 5s 517us/step - loss: 0.0247 - acc: 0.8356 - val_loss: 0.0170 - val_acc: 0.8822
Epoch 991/1000
8755/8755 [==============================] - 5s 522us/step - loss: 0.0247 - acc: 0.8358 - val_loss: 0.0169 - val_acc: 0.8823
Epoch 992/1000
8755/8755 [==============================] - 5s 523us/step - loss: 0.0247 - acc: 0.8357 - val_loss: 0.0169 - val_acc: 0.8781
Epoch 993/1000
8755/8755 [==============================] - 5s 521us/step - loss: 0.0250 - acc: 0.8361 - val_loss: 0.0170 - val_acc: 0.8826
Epoch 994/1000
8755/8755 [==============================] - 5s 530us/step - loss: 0.0249 - acc: 0.8336 - val_loss: 0.0169 - val_acc: 0.8749
Epoch 995/1000
8755/8755 [==============================] - 5s 525us/step - loss: 0.0247 - acc: 0.8313 - val_loss: 0.0169 - val_acc: 0.8919
Epoch 996/1000
8755/8755 [==============================] - 5s 522us/step - loss: 0.0249 - acc: 0.8307 - val_loss: 0.0169 - val_acc: 0.8777
Epoch 997/1000
8755/8755 [=========

In [36]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          1800      
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 22,000
Trainable params: 22,000
Non-trainable params: 0
_________________________________________________________________


In [37]:
encoder_inputs,encoder_states

(<tf.Tensor 'input_1:0' shape=(None, None) dtype=float32>,
 [<tf.Tensor 'lstm_1/while/Identity_4:0' shape=(None, 50) dtype=float32>,
  <tf.Tensor 'lstm_1/while/Identity_5:0' shape=(None, 50) dtype=float32>])

## Create Sampling Model

In [38]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [39]:
# reverse_input_char_index,reverse_target_char_index

# Function to generate sequences

In [40]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['start_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_end' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [41]:
encoder_input_data.shape

(9216, 8)

In [42]:
## Predictions
for seq_index in [100,101,102,113,133]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', sequenceData[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['past three hour walk status ']
Decoded sentence:  select walk where hour is sum of past three hour _end
-
Input sentence: ['last three hour walk status ']
Decoded sentence:  select walk where hour is sum of past three hours _end
-
Input sentence: ['past four hour walk status ']
Decoded sentence:  select walk where hour is sum of past four hour _end
-
Input sentence: ['last nine hour walk status ']
Decoded sentence:  select walk where hour is sum of past nine hours _end
-
Input sentence: ['last two days walk status ']
Decoded sentence:  select walk where hour is sum of past forty eigthhours _end


In [54]:
input_sentence = "past six hour walk status"

test_sentence_tokenized = np.zeros((total_length_seq, max_seq_length),dtype='float32')
print(test_sentence_tokenized.shape)

for t, word in enumerate(input_sentence.split()):
    test_sentence_tokenized[0, t] = input_token_index[word]
    
print("\n")   
print(input_sentence,"-->\n",decode_sequence(test_sentence_tokenized))

(9216, 8)


past six hour walk status -->
  select walk where hour is sum of past six hour _end
